# Subclassing ndarray

https://docs.scipy.org/doc/numpy-1.13.0/user/basics.subclassing.html

In [1]:
%numpy

Numpy 1.14.0


## `__new__` and `__array_finalize__`

`self` is always a newly created instance of our subclass, and the type of `obj` differs for the three instance creation methods:

- When called from the explicit constructor, `obj` is None
- When called from view casting, `obj` can be an instance of any subclass of `ndarray`, including our own.
- When called in new-from-template, `obj` is another instance of our own subclass, that we might use to update the new `self` instance.

In [7]:
class C(np.ndarray):
    def __new__(cls, *args, **kwargs):
        print('In __new__ with class %s' % cls)
        return super(C, cls).__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        # in practice you probably will not need or want an __init__
        # method for your subclass
        print('In __init__ with class %s' % self.__class__)

    def __array_finalize__(self, obj):
        print('In array_finalize:')
        print('   self type is %s' % type(self))
        print('   obj type is %s' % type(obj))

In [10]:
# Explicit constructor
C((10,))

In __new__ with class <class '__main__.C'>
In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class 'NoneType'>
In __init__ with class <class '__main__.C'>


C([4.63586546e-310, 6.79038654e-313, 6.79038653e-313, 2.37663529e-312,
   2.58883487e-312, 2.41907520e-312, 2.44029516e-312, 8.48798317e-313,
   9.33678148e-313, 8.70018274e-313])

In [11]:
# View
arr = np.zeros((3,))
c_arr = arr.view(C)
c_arr

In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class 'numpy.ndarray'>


C([0., 0., 0.])

In [12]:
c_arr[1:]

In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class '__main__.C'>


C([0., 0.])

In [13]:
c_arr.copy()

In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class '__main__.C'>


C([0., 0., 0.])

In [14]:
c_arr + c_arr

In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class '__main__.C'>


C([0., 0., 0.])

In [15]:
class InfoArray(np.ndarray):

    def __new__(subtype, shape, dtype=float, buffer=None, offset=0,
                strides=None, order=None, info=None):
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        obj = super(InfoArray, subtype).__new__(subtype, shape, dtype,
                                                buffer, offset, strides,
                                                order)
        # set the new 'info' attribute to the value passed
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None: return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).
        self.info = getattr(obj, 'info', None)
        # We do not need to return anything


In [16]:
obj = InfoArray(shape=(3,)) # explicit constructor
obj, obj.info

(InfoArray([0., 0., 0.]), None)

In [17]:
obj = InfoArray(shape=(3,), info='information')
obj, obj.info

(InfoArray([0., 0., 0.]), 'information')

In [18]:
v = obj[1:] # new-from-template - here - slicing
v, v.info

(InfoArray([0., 0.]), 'information')

In [19]:
arr = np.arange(10)
cast_arr = arr.view(InfoArray) # view casting
cast_arr, cast_arr.info

(InfoArray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), None)

## `__array_ufunc__` for ufuncs

```
def __array_ufunc__(ufunc, method, *inputs, **kwargs):

- *ufunc* is the ufunc object that was called.
- *method* is a string indicating how the Ufunc was called, either
  ``"__call__"`` to indicate it was called directly, or one of its
  :ref:`methods<ufuncs.methods>`: ``"reduce"``, ``"accumulate"``,
  ``"reduceat"``, ``"outer"``, or ``"at"``.
- *inputs* is a tuple of the input arguments to the ``ufunc``
- *kwargs* contains any optional or keyword arguments passed to the
  function. This includes any ``out`` arguments, which are always
  contained in a tuple.
```

In [20]:
class A(np.ndarray):
    def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
        args = []
        in_no = []
        for i, input_ in enumerate(inputs):
            if isinstance(input_, A):
                in_no.append(i)
                args.append(input_.view(np.ndarray))
            else:
                args.append(input_)

        outputs = kwargs.pop('out', None)
        out_no = []
        if outputs:
            out_args = []
            for j, output in enumerate(outputs):
                if isinstance(output, A):
                    out_no.append(j)
                    out_args.append(output.view(np.ndarray))
                else:
                    out_args.append(output)
            kwargs['out'] = tuple(out_args)
        else:
            outputs = (None,) * ufunc.nout

        info = {}
        if in_no:
            info['inputs'] = in_no
        if out_no:
            info['outputs'] = out_no

        results = super(A, self).__array_ufunc__(ufunc, method,
                                                 *args, **kwargs)
        if results is NotImplemented:
            return NotImplemented

        if method == 'at':
            if isinstance(inputs[0], A):
                inputs[0].info = info
            return

        if ufunc.nout == 1:
            results = (results,)

        results = tuple((np.asarray(result).view(A)
                         if output is None else output)
                        for result, output in zip(results, outputs))
        if results and isinstance(results[0], A):
            results[0].info = info

        return results[0] if len(results) == 1 else results

In [21]:
>>> a = np.arange(5.).view(A)
>>> b = np.sin(a)
>>> b.info

{'inputs': [0]}